# Importing the Dependencies

In [62]:
import pandas as pd  
import numpy as np  
import os
import json
import jsonify
import traceback
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI

In [63]:
load_dotenv() # Loading the env to fetch environment variables

KEY = os.getenv("OPENAI_API_KEY")

In [64]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.3)

# Importing the required libraries and modules from Langchain

In [65]:
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains import LLMChain 
from langchain.chains import SequentialChain 
from langchain.callbacks import get_openai_callback 
import PyPDF2

# Quiz Generation - Prompt and Chain Initialization

In [66]:
RESPONSE_JSON = { # Here, we will just store the correct answer. Later, we can do a direct string match
    "1": {
        "fbq": "fill in the blank question",
        "correct": "correct answer",
    },
    "3": {
        "fbq": "fill in the blank question",
        "correct": "correct answer",
    },
    "3": {
        "fbq": "fill in the blank question",
        "correct": "correct answer",
    }
}

In [74]:
TEMPLATE = """
Text: {text}
You are an expert Fill in the Blank Question (FBQs) maker. Given the above text/formulas, it is your job to \
create a quiz of {number} Fill in the Blank Question for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. Basically, it be questions based on numeric 
formulas and you could use dummy values and give the same to the user. In the end, the user will just give the final answer and 
that will be string matched with the correct answer. So for example if you are given a formula in text that a^2 + b^2 = c^2, 
the question will be what is the value of a^2 + b^2, given that a and b are some constants (you can choose those and tell them to the user)
and store the answer of the above correct answer. Make sure that the correct answer is a singular value only, as it has to be string matches. If any question 
may have two or more answers, kindly choose the minimum one\
Ensure to make {number} FBQs

### RESPONSE_JSON
{response_json}
"""

# Here, tone implies difficulty level



quizGen_prompt_template = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)


quiz_chain = LLMChain(llm=llm,prompt=quizGen_prompt_template,output_key="quiz",verbose=True)

# Answer Evalusation Template Initiliazation

In [75]:
TEMPLATE2="""
You are an expert math. Given a Fill in the Blanks Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_FBQs:
{quiz}

Check from an expert Math Expert of the above quiz:
"""

quizEval_prompt_template = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

review_chain = LLMChain(llm=llm,prompt=quizEval_prompt_template,output_key="review",verbose=True)

# Sequential Chain Generation
Let us now generate a chain of prompts and answers for the quiz generation task. Here, both the above chains will be combined to generate a single chain and give us the final output as desired.

In [76]:
evaluation_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                        output_variables=["quiz","review"],verbose=True)

# Data Loading

In [77]:
file_path = '../formula_data.txt'
with open(file_path,'r') as file:
    TEXT = file.read()

print(TEXT)

1. Quadratic Formula: ( ax^2 + bx + c = 0 )
2. Pythagorean Theorem: ( a^2 + b^2 = c^2 )
3. Area of a Circle: ( A = pi r^2 )
4. Area of a Triangle: ( A = \frac{{1}}{{2}}b*h )
5. Volume of a Sphere: ( V = frac{{4}}{{3}}pi r^3 )
6. Perimeter of a Rectangle: ( P = 2l + 2w )
7. Equation of a Line (Point-Slope Form): ( y - y_1 = m(x - x_1) )



In [78]:
# Now, let us serialize the above python dictionary into a json-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"fbq": "fill in the blank question", "correct": "correct answer"}, "3": {"fbq": "fill in the blank question", "correct": "correct answer"}}'

# Callback Function for Quiz Generation
Now, we will define a callback function that will be used to generate the quiz questions and answers. This function will be called by the chain to generate the quiz questions and answers. By this, we can track the tokens used in the prompt and the answer and get the cost of the prompt and the answer. Read more here [here](https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking).

In [79]:
NUMBER = 5
SUBJECT = "Math Quiz"
TONE = "Simple"

In [80]:
with get_openai_callback() as cb:
    response=evaluation_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: 1. Quadratic Formula: ( ax^2 + bx + c = 0 )
2. Pythagorean Theorem: ( a^2 + b^2 = c^2 )
3. Area of a Circle: ( A = pi r^2 )
4. Area of a Triangle: ( A = \frac{{1}}{{2}}b*h )
5. Volume of a Sphere: ( V = frac{{4}}{{3}}pi r^3 )
6. Perimeter of a Rectangle: ( P = 2l + 2w )
7. Equation of a Line (Point-Slope Form): ( y - y_1 = m(x - x_1) )

You are an expert Fill in the Blank Question (FBQs) maker. Given the above text/formulas, it is your job to create a quiz of 5 Fill in the Blank Question for Math Quiz students in Simple tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. Basically, it be questions based on numeric 
formulas and you could use dummy values and give the same to the user. In the end, the user will just give the final answer and 

Let us get the token usage of the prompt and the answer.

In [81]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost in USD:{cb.total_cost}")

Total Tokens:1074
Prompt Tokens:794
Completion Tokens:280
Total Cost in USD:0.001751


Let us now try to get the response from the chain and see our quiz.

In [82]:
response

{'text': '1. Quadratic Formula: ( ax^2 + bx + c = 0 )\n2. Pythagorean Theorem: ( a^2 + b^2 = c^2 )\n3. Area of a Circle: ( A = pi r^2 )\n4. Area of a Triangle: ( A = \\frac{{1}}{{2}}b*h )\n5. Volume of a Sphere: ( V = frac{{4}}{{3}}pi r^3 )\n6. Perimeter of a Rectangle: ( P = 2l + 2w )\n7. Equation of a Line (Point-Slope Form): ( y - y_1 = m(x - x_1) )\n',
 'number': 5,
 'subject': 'Math Quiz',
 'tone': 'Simple',
 'response_json': '{"1": {"fbq": "fill in the blank question", "correct": "correct answer"}, "3": {"fbq": "fill in the blank question", "correct": "correct answer"}}',
 'quiz': '{\n"1": {"fbq": "What is the value of x in the quadratic formula ax^2 + bx + c = 0, given a = 2, b = 3, and c = 1?", "correct": "-0.5"},\n"2": {"fbq": "In the Pythagorean theorem a^2 + b^2 = c^2, if a = 3 and c = 5, what is the value of b?", "correct": "4"},\n"3": {"fbq": "Find the area of a circle with radius r = 4. Use the formula A = pi r^2.", "correct": "50.24"},\n"4": {"fbq": "Calculate the area o

In [83]:
quiz = response.get('quiz')
quiz = json.loads(quiz)
quiz

{'1': {'fbq': 'What is the value of x in the quadratic formula ax^2 + bx + c = 0, given a = 2, b = 3, and c = 1?',
  'correct': '-0.5'},
 '2': {'fbq': 'In the Pythagorean theorem a^2 + b^2 = c^2, if a = 3 and c = 5, what is the value of b?',
  'correct': '4'},
 '3': {'fbq': 'Find the area of a circle with radius r = 4. Use the formula A = pi r^2.',
  'correct': '50.24'},
 '4': {'fbq': 'Calculate the area of a triangle with base b = 6 and height h = 8. Use the formula A = 1/2*b*h.',
  'correct': '24'},
 '5': {'fbq': 'Determine the volume of a sphere with radius r = 3. Use the formula V = 4/3*pi*r^3.',
  'correct': '113.04'}}

Let us now convert the response to a DataFrame and display the quiz.

In [86]:
quiz_table_data = []
for key, value in quiz.items():
    tfq = value["fbq"]
    correct = value["correct"]
    quiz_table_data.append({"FBQ": tfq, "Correct": correct})

In [87]:
quiz = pd.DataFrame(quiz_table_data)
quiz

,FBQ,Correct
0,What is the value of x in the quadratic formul...,-0.5
1,"In the Pythagorean theorem a^2 + b^2 = c^2, if...",4
2,Find the area of a circle with radius r = 4. U...,50.24
3,Calculate the area of a triangle with base b =...,24
4,Determine the volume of a sphere with radius r...,113.04


Now finally, we can export the above DataFrame to a csv file.

In [88]:
quiz.to_csv("ML_FBQ_Quiz.csv",index= False)